In [3]:
import pyterrier as pt
import pandas as pd
import pickle
import nltk
import numpy as np

In [4]:
pd.options.mode.copy_on_write = True

In [5]:
df_jahreswechsel = pd.read_json('jahreswechsel.json')

In [6]:
df_jahreswechsel

,id,tags,title,user,description,date,changeDate,url,pub-type,year,...,volume,pages,bibtexKey,pid,booktitle,series,publisher,editors,isbn,search_term
0,https://www.bibsonomy.org/bibtex/2a9f754572c68...,"[Rom, Silvester]","Silvester I., römischer Bischof",avs,,2009-01-20 09:50:15,2009-01-20 09:50:16,,article,2004,...,VII,1321--1322,2004b,None,None,None,None,None,None,Silvester
1,https://www.bibsonomy.org/bibtex/241dfc656397b...,"[Purchasing, the, Price, and, Decision, Promot...",The Effect of Price and Promotion on the Purch...,ijtsrd,,2024-03-08 08:00:25,2024-03-08 08:00:25,https://www.ijtsrd.com/economics/market-econom...,article,2024,...,8,636-657,HastieEtAl2008,None,None,None,None,None,None,Silvester
2,https://www.bibsonomy.org/bibtex/235bf77da240d...,"[Location,, Rates,, Cooperative, Service, the,...","The Influence of Location, Credit Interest Rat...",ijtsrd,,2024-03-08 08:02:00,2024-03-08 08:02:00,https://www.ijtsrd.com/economics/development-e...,article,2024,...,8,658-675,journals/is/Formica03,http://dx.doi.org/10.1016/S0306-4379(02)00010-8,None,None,None,None,None,Silvester
3,https://www.bibsonomy.org/bibtex/23674c2f2342a...,[imported],"Silvester I., r�mischer Bischof",avs,,2009-01-20 09:49:38,2009-01-20 09:49:40,,article,2004,...,VII,1321--1322,2004b,None,None,None,None,None,None,Silvester
4,https://www.bibsonomy.org/bibtex/289e0b4f233d1...,[dnb],Gerbert (Silvester II.) als Persönlichkeit,genealogie,,2016-05-31 03:35:04,2016-05-31 03:35:04,,phdthesis,1928,...,None,None,eichengrun1928gerbert,None,None,None,None,None,None,Silvester
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,https://www.bibsonomy.org/bibtex/209424eb679de...,[dblp],Solving Multidimensional 0-1 Knapsack Problem ...,dblp,,2018-11-14 00:00:00,2019-05-15 17:34:36,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7335,72-86,conf/iccsa/AzadRF12,https://www.wikidata.org/entity/Q57573541,ICCSA (3),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31136-9,Neujahr
1953,https://www.bibsonomy.org/bibtex/210805c159439...,[dblp],Axis-Parallel Dimension Reduction for Biometri...,dblp,,2017-05-24 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,188-197,conf/iccsa/AhmadianG12,https://doi.org/10.1007/978-3-642-31125-3_15,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr
1954,https://www.bibsonomy.org/bibtex/2935b7a107d2a...,[dblp],A Review on Delaunay Refinement Techniques.,dblp,,2017-05-24 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,172-187,conf/iccsa/Oliveira12,https://doi.org/10.1007/978-3-642-31125-3_14,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr
1955,https://www.bibsonomy.org/bibtex/29432427bc781...,[dblp],Theoretical and Experimental Study of the Ener...,dblp,,2018-11-02 00:00:00,2019-05-15 17:32:59,http://dblp.uni-trier.de/db/conf/iccsa/iccsa20...,inproceedings,2012,...,7333,316-330,conf/iccsa/RosiCFMPV12,https://doi.org/10.1007/978-3-642-31125-3_25,ICCSA (1),Lecture Notes in Computer Science,Springer,"[Beniamino Murgante, Osvaldo Gervasi, Sanjay M...",978-3-642-31124-6,Neujahr


In [7]:
df_jahreswechsel.shape

(1957, 21)

In [8]:
df_jahreswechsel_filtered = df_jahreswechsel.dropna(axis=1, thresh=len(df_jahreswechsel)*0.5)

In [9]:
df_jahreswechsel_filtered.shape

(1957, 19)

In [10]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'isbn', 'search_term'],
      dtype='object')

In [11]:
df_jahreswechsel_filtered = df_jahreswechsel_filtered.drop(columns=['isbn'])

In [12]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'search_term'],
      dtype='object')

In [13]:
if not pt.java.started():
    pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_81848/2059753628.py:2: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [14]:
df_jahreswechsel_filtered['docno'] = df_jahreswechsel_filtered['id']
df_jahreswechsel_filtered['text'] = df_jahreswechsel_filtered['title']

In [15]:
df_jahreswechsel_filtered.columns

Index(['id', 'tags', 'title', 'user', 'description', 'date', 'changeDate',
       'url', 'pub-type', 'year', 'authors', 'pages', 'bibtexKey', 'pid',
       'booktitle', 'publisher', 'editors', 'search_term', 'docno', 'text'],
      dtype='object')

In [16]:
jw_dict = df_jahreswechsel_filtered.to_dict(orient='records')

In [17]:
index_folder = './jw_index_mult'

#index_fields = ['docno', 'text', 'tags', 'user', 'description', 'bibtexKey', 'booktitle']
index_fields = ['docno', 'text', 'tags', 'user', 'description',
       'url', 'authors', 'bibtexKey', 'publisher', 'editors',]

indexer = pt.IterDictIndexer(index_folder, meta={'docno': 200,'text': 4096}, overwrite=True)
index_ref = indexer.index(jw_dict, fields=index_fields)

print(f'Indexing-Folder ist {index_folder}.')

Indexing-Folder ist ./jw_index_mult.


In [18]:
index = pt.IndexFactory.of(index_ref)

In [19]:
print(index.getCollectionStatistics().toString())

Number of documents: 1957
Number of terms: 13001
Number of postings: 66492
Number of fields: 10
Number of tokens: 76183
Field names: [docno, text, tags, user, description, url, authors, bibtexKey, publisher, editors]
Positions:   false



In [20]:
for kv in index.getLexicon():
    print(f"{kv.getKey()} --> {kv.getValue().toString()}")

0 --> term545 Nt=16 TF=29 maxTF=4 @{0 0 0} TFf=0,9,0,0,0,20,0,0,0,0
00 --> term1574 Nt=11 TF=11 maxTF=1 @{0 45 0} TFf=0,0,0,0,0,11,0,0,0,0
0001 --> term1770 Nt=2 TF=2 maxTF=1 @{0 71 1} TFf=0,0,0,0,0,2,0,0,0,0
001 --> term1771 Nt=2 TF=2 maxTF=1 @{0 76 5} TFf=0,0,0,0,0,2,0,0,0,0
0018 --> term12863 Nt=1 TF=2 maxTF=2 @{0 82 1} TFf=1,0,0,0,0,1,0,0,0,0
002 --> term7290 Nt=1 TF=1 maxTF=1 @{0 86 4} TFf=0,0,0,0,0,1,0,0,0,0
01 --> term1762 Nt=1 TF=1 maxTF=1 @{0 90 3} TFf=0,0,1,0,0,0,0,0,0,0
01a55 --> term2675 Nt=1 TF=1 maxTF=1 @{0 93 6} TFf=0,0,1,0,0,0,0,0,0,0
01a60 --> term2672 Nt=1 TF=1 maxTF=1 @{0 97 1} TFf=0,0,1,0,0,0,0,0,0,0
03 --> term2666 Nt=1 TF=1 maxTF=1 @{0 100 4} TFf=0,0,1,0,0,0,0,0,0,0
0427 --> term1575 Nt=1 TF=1 maxTF=1 @{0 103 7} TFf=0,0,0,0,0,1,0,0,0,0
0812 --> term7544 Nt=1 TF=1 maxTF=1 @{0 107 2} TFf=0,0,0,0,0,1,0,0,0,0
0909 --> term5630 Nt=1 TF=1 maxTF=1 @{0 111 1} TFf=0,0,0,0,0,1,0,0,0,0
0954 --> term7287 Nt=1 TF=1 maxTF=1 @{0 115 0} TFf=0,0,0,0,0,1,0,0,0,0
0auth2 --> term1106

In [21]:
term_dict = {}

for kv in index.getLexicon():
    term_dict[kv.getKey()] = kv.getValue().frequency


In [22]:
term_dict_ordered = sorted(term_dict.items(), key= lambda x: x[1], reverse=True)

In [23]:
top_k = 50

term_dict_ordered[:top_k]

[('dblp', 6325),
 ('http', 3630),
 ('de', 2274),
 ('org', 2018),
 ('www', 2014),
 ('bibsonomi', 1967),
 ('bibtex', 1956),
 ('null', 1573),
 ('db', 1572),
 ('html', 1571),
 ('uni', 1562),
 ('trier', 1561),
 ('conf', 1231),
 ('ieee', 597),
 ('nedjah', 502),
 ('nadia', 500),
 ('luiza', 452),
 ('mourel', 452),
 ('macedo', 451),
 ('genealogi', 404),
 ('sylvest', 399),
 ('comput', 392),
 ('societi', 346),
 ('springer', 309),
 ('journal', 294),
 ('abraham', 279),
 ('ajith', 278),
 ('silvest', 228),
 ('system', 207),
 ('dnb', 202),
 ('silvestr', 198),
 ('und', 193),
 ('us', 192),
 ('base', 188),
 ('neumair', 183),
 ('network', 177),
 ('david', 175),
 ('bernhard', 171),
 ('paul', 168),
 ('model', 166),
 ('janusz', 161),
 ('kacprzyk', 161),
 ('ller', 156),
 ('isda', 155),
 ('isda2007', 155),
 ('der', 153),
 ('gabi', 151),
 ('dreo', 150),
 ('rodosek', 149),
 ('gi', 146)]

In [24]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [25]:
def get_idf_for_term(term, index):
  lex = index.getLexicon()
  stemmed_term = stemmer.stem(term)

  if not stemmed_term in lex:
    return 
    
  lex_entry = lex[stemmed_term]

  df_term = lex_entry.getDocumentFrequency()
  N = index.getCollectionStatistics().numberOfDocuments
  
  #inverse document frequency
  idf = N/df_term

  #apply logarithm(base 10) to idf
  log_idf = np.emath.logn(7, idf)

  return log_idf

In [26]:
get_idf_for_term("learning", index)

1.6900618601759512

In [31]:
#define your own tf_idf method here:

def calc_tf_idf(query, docno, index):

    entry_list = query.split(" ")
    tf_idf_list = []
    for entry in entry_list:
        print(entry)
        search_engine_tmp = pt.BatchRetrieve(index, wmodel="Tf")
        df_query = search_engine_tmp.search(entry)
    
        tf = df_query[["score"]].loc[df_query["docno"] == docno]
    
        print(f"TF von {entry} im Dokument {docno} ist {str(tf)}.")

        idf =  get_idf_for_term(query, index)
        print(f"inverse Dokumentfrequenz ist {idf}")

        tf_idf = tf*idf
        print(f"TF_IDF von {entry} ist {tf_idf}.") 
        tf_idf_list.append(tf_idf)
    #remember that tfidf is the product of two components
    #hint: the tf model search result contains tf frequencies
    #return tf_idf

calc_tf_idf("learning round", "https://www.bibsonomy.org/bibtex/2a6cabc9bcf528ed144b490437f4e06bb/dblp", index)

learning
TF von learning im Dokument https://www.bibsonomy.org/bibtex/2a6cabc9bcf528ed144b490437f4e06bb/dblp ist    score
1    2.0.
inverse Dokumentfrequenz ist None


/tmp/ipykernel_81848/1727190915.py:9: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tmp = pt.BatchRetrieve(index, wmodel="Tf")


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [129]:
search_engine_tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

/tmp/ipykernel_43742/3523076512.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")


In [130]:
search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")

/tmp/ipykernel_43742/3266775375.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")


In [131]:
pd.set_option('display.max_colwidth', None)

In [132]:
search_engine_tf_idf.search("learning")[["docno","score"]]

,docno,score
0,https://www.bibsonomy.org/bibtex/2a6cabc9bcf528ed144b490437f4e06bb/dblp,3.399819
1,https://www.bibsonomy.org/bibtex/2e856db87128141f1fe74deca12e22dd0/dblp,3.309973
2,https://www.bibsonomy.org/bibtex/2cdd1105fa067948608b52272b0a23cd2/dblp,3.309448
3,https://www.bibsonomy.org/bibtex/2862b1c4ff0ad5761c098c72740fa2297/tomp,3.224255
4,https://www.bibsonomy.org/bibtex/26c35a13d75db76c053bf1ffc62508812/ijtsrd,3.203024
...,...,...
68,https://www.bibsonomy.org/bibtex/281594c96df2619f5b4caaf9b5cd4e98f/dblp,2.238688
69,https://www.bibsonomy.org/bibtex/27cfa762b4d610588ad3b49bbe8412e6b/dblp,2.218741
70,https://www.bibsonomy.org/bibtex/21d0f9519908930ffab4a8d3b6c903eeb/dblp,2.218741
71,https://www.bibsonomy.org/bibtex/270b9e55ad70ff491fdf7df5623599ba8/dblp,2.199146


In [133]:
search_engine_tf.search("learning")

,qid,docid,docno,rank,score,query
0,1,249,https://www.bibsonomy.org/bibtex/26c35a13d75db76c053bf1ffc62508812/ijtsrd,0,2.0,learning
1,1,1799,https://www.bibsonomy.org/bibtex/2a6cabc9bcf528ed144b490437f4e06bb/dblp,1,2.0,learning
2,1,1830,https://www.bibsonomy.org/bibtex/2cdd1105fa067948608b52272b0a23cd2/dblp,2,2.0,learning
3,1,89,https://www.bibsonomy.org/bibtex/29d93fb3056a9faccc30e0ab5689ecd4c/dblp,3,1.0,learning
4,1,96,https://www.bibsonomy.org/bibtex/2b2dd6dcc89ca2147726f05707a6d4205/dblp,4,1.0,learning
...,...,...,...,...,...,...
68,1,1856,https://www.bibsonomy.org/bibtex/28be0429556f638898a27820841d447de/dblp,68,1.0,learning
69,1,1864,https://www.bibsonomy.org/bibtex/2975bfdb8de7432ed35925c4ca53510d6/dblp,69,1.0,learning
70,1,1902,https://www.bibsonomy.org/bibtex/2327000aa40c32b48206526c4f69a1199/dblp,70,1.0,learning
71,1,1907,https://www.bibsonomy.org/bibtex/2369046d8447497a781319e788d64dba5/dblp,71,1.0,learning


In [136]:
pickle.dump(df_jahreswechsel_filtered, open("./jw_index_mult/jw_publications.pkl", "wb"))